Things to note: 
 - There are a lot of unnamed parameters if you investigate, but simply zeroing out the unnamed parameters kills the patch, so they must be used somehow
 - Some of these settings are binary or categorical, so I'm not sure how they are interacting with the float values.

In [54]:
import librenderman as rm
import gzip
import cPickle

In [55]:
#Set this to wherever you have autosynth saved
autosynth_dir = "/Users/samlowe/autosynth"

#Set how many samples to generate
num_samples = 100

In [56]:
# Important settings. These are good general ones.
sampleRate = 44100
bufferSize = 512
fftSize = 512

# Settings to play a note and extract data from the synth.
midiNote = 40
midiVelocity = 127
noteLength = 4.0
renderLength = 5.0

In [57]:
# This will host a VST. It will render the features and audio we need.
engine = rm.RenderEngine(sampleRate, bufferSize, fftSize)

# Load the VST into the RenderEngine.
path = autosynth_dir + "/Synths/Synth1.vst"
engine.load_plugin(path)

# Create a patch generator. We can initialise it to generate the correct
# patches for a given synth by passing it a RenderEngine which has
# loaded a instance of the synthesiser. 
generator = rm.PatchGenerator(engine)

In [58]:
samples = []

for i in range(num_samples):
    new_patch = generator.get_random_patch()
    engine.set_patch(new_patch)

    # Render the data. 
    engine.render_patch(midiNote, midiVelocity, noteLength, renderLength)

    # Get the data. Note the audio is automattically made mono, no
    # matter what channel size for ease of use.
    audio = engine.get_audio_frames()
    
    sample = (audio, new_patch)
    samples.append(sample)

In [ ]:
datafile = gzip.open('patches.data','wb')
cPickle.dump(samples, datafile)
datafile.close()